In [1]:
pip install pandas requests


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade TikTokApi

In [ ]:
pip install --upgrade TikTokApi playwright pandas nest_asyncio
playwright install

In [4]:
import requests
import pandas as pd

# Configuration
RAPIDAPI_KEY = "f1748cf772mshf37bd83f9aa9cf2p115781jsn8b48ae67e28d"   #change to your key
RAPIDAPI_HOST = "tiktok-api23.p.rapidapi.com"
BASE_URL = "https://tiktok-api23.p.rapidapi.com/api/post/trending"

def fetch_multiple_batches(max_results=500):
    """Fetch multiple batches to get more than 8 results"""
    all_items = []
    cursor = "0"

    while len(all_items) < max_results:
        headers = {
            "X-RapidAPI-Host": RAPIDAPI_HOST,
            "X-RapidAPI-Key": RAPIDAPI_KEY
        }

        params = {
            "count": 8,
            "cursor": cursor
        }

        try:
            response = requests.get(BASE_URL, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            if not data.get('itemList'):
                break

            all_items.extend(data['itemList'])
            cursor = data.get('cursor', "0")

            # Stop if no more items available
            if not data.get('hasMore', False):
                break

        except Exception as e:
            print(f"Error fetching batch: {e}")
            break

    return all_items[:max_results]

def process_data(items):
    processed = []
    for item in items:
        processed.append({
            "video_id": item.get('id'),
            "author": item.get('author', {}).get('uniqueId'),
            "description": item.get('desc'),
            "likes": item.get('stats', {}).get('diggCount'),
            "comments": item.get('stats', {}).get('commentCount'),
            "shares": item.get('stats', {}).get('shareCount'),
            "plays": item.get('stats', {}).get('playCount'),
            "hashtags": ", ".join([h['title'] for h in item.get('challenges', [])]),
            "music": item.get('music', {}).get('title'),
            "create_time": pd.to_datetime(item.get('createTime'), unit='s') if item.get('createTime') else None,
            "video_url": f"https://www.tiktok.com/@{item.get('author', {}).get('uniqueId')}/video/{item.get('id')}"
        })
    return pd.DataFrame(processed)

# Fetch data
print("Fetching trending TikTok videos...")
items = fetch_multiple_batches(max_results=500)
df = process_data(items)

print(f"\nSuccessfully fetched {len(df)} videos")
display(df.head(10))
df.to_csv('tiktok_trending26_eq.csv', index=False)     #rename your file
print("\nData saved to 'tiktok_trending26_eq.csv'")

Fetching trending TikTok videos...
Error fetching batch: 429 Client Error: Too Many Requests for url: https://tiktok-api23.p.rapidapi.com/api/post/trending?count=8&cursor=0

Successfully fetched 0 videos


""



Data saved to 'tiktok_trending26_eq.csv'
